In [43]:
import xml.etree.ElementTree as ET
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Alignment, Border, Side, Font

# Parse the input XML file
tree = ET.parse('/content/inputxmlFile.xml')
root = tree.getroot()

# Create a new workbook and get the active sheet
workbook = Workbook()
sheet = workbook.active

def get_data(path, name, is_period = False):
  elem_dict = {}
  for elem in root.findall('.//'+ path + '//'):
    if is_period == False:
      elem_dict[elem.attrib.get('id')] = elem.attrib.get(name)
    else:
      elem_dict[elem.attrib.get(name)] = elem.attrib.get('starttime') + ' to ' + elem.attrib.get('endtime')
  return elem_dict

def remove_from_dict(elem_dict, arr):
  new_dict = {}
  for key, value in (elem_dict.items()):
    if value in arr:
      new_dict[key] = value
  return new_dict

def create_header_with_style(row_header, column_header, header_color, header_alignment, font_size):
  for col_num, header in enumerate(column_header, 2):
    col_letter = get_column_letter(col_num)
    cell = sheet[f"{col_letter}1"]
    cell.value = header

    # Apply green fill color to the header row
    cell.fill = PatternFill(start_color=header_color, end_color=header_color, fill_type="solid")
    cell.alignment = Alignment(horizontal=header_alignment, vertical=header_alignment)
    cell.font = Font(bold=True, size=font_size)

  for row_num, header in enumerate(row_header, 2):
    cell = sheet[f"A{row_num}"]
    cell.value = header
    cell.fill = PatternFill(start_color=header_color, end_color=header_color, fill_type="solid")
    cell.alignment = Alignment(horizontal=header_alignment, vertical=header_alignment)
    cell.font = Font(bold=True, size=font_size)

  # Adjust column width and row height to fit content
  for column_cells in sheet.columns:
    max_length = 0
    column = column_cells[0].column_letter
    for cell in column_cells:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except TypeError:
            pass
    adjusted_width = (max_length + 2) * 1.4
    sheet.column_dimensions[column].width = adjusted_width

  # Apply dark border to the entire table
  border_style = Side(border_style="thick", color="000000")  # Thick and black border

  for row in sheet.iter_rows(min_row=1, max_row=len(row_header)+1):
    max_height = 0
    for cell in row:
        try:
            cell.border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
            if len(str(cell.value)) > max_height:
                max_height = len(str(cell.value))
        except TypeError:
            pass
    adjusted_height = (max_height + 2) * 4
    sheet.row_dimensions[cell.row].height = adjusted_height


faculty_dict = get_data('teachers', 'name')
class_dict = get_data('classes', 'name')
day_dict = get_data('daysdefs', 'name')
period_dict = get_data('periods', 'name', True)

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
day_dict = remove_from_dict(day_dict, days)

create_header_with_style(day_dict.values(), period_dict.values(), '00FF00', 'center', 14)



# Remove the default sheet created by openpyxl
#default_sheet = workbook["Sheet"]
#workbook.remove(default_sheet)

# Save the output Excel file
workbook.save("Output_Excel File.xlsx")
